### 様々な入力からツイートデータを収集してCSVに書き起こすプログラム

In [25]:
import tweepy
from datetime import datetime,timezone
import pytz
import pandas as pd
import os
from dotenv import load_dotenv

##### Twitter認証

In [26]:
# Twitter情報

load_dotenv()

# API情報
CK = os.environ['CONSUMER_KEY']
CS = os.environ['CONSUMER_SECRET']
AT = os.environ['ACCESS_TOKEN']
ATS = os.environ['ACCESS_TOKEN_SECRET']

#Twitterの認証
auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AT, ATS)
api = tweepy.API(auth)

#　”wait_on_rate_limit = True”　利用制限にひっかかた時に必要時間待機する
api=tweepy.API(auth,wait_on_rate_limit=True)

##### ↓次のセルを変更
- search_word...検索したいキーワードを指定
- req_tweet_cnt...取得するツイート数を指定
  - 450 requests / 15 mins PER APP
  - 180 requests / 15 mins PER USER

In [27]:
#取得するツイートの数と、検索するアカウントの入力
search_word = 'from:@sakuramiko35 -filter:retweets -filter:replies -filter:links'
req_tweet_cnt = 1000

##### 必要なTweetデータを取得
api.user_timeline： 特定のユーザーのタイムライン情報を取得

``` python
tweepy.Cursor(api.search_tweets,q=search_word, tweet_mode='extended',result_type="mixed",lang='ja').items(req_tweet_cnt)
```

|search_word|説明|
|:---:|:---:|
|キーワード -除外するキーワード|	-の後に続くキーワードを除外して検索|
|キーワード min_faves:100|	いいねの数が100以上のツイートだけ|
|キーワード min_retweets:100|	100リツイート以上のツイートを検索|
|from:ユーザーネーム|	特定のユーザーのツイートを検索|
|キーワード1 OR キーワード2|	ブログもしくはブロガーのキーワードを検索|
|“キーワード”|	” “で囲われた文字列に完全一致するツイートを検索|

<br/>

|result_type|説明|
|:---:|:---:|
|recent|最新のツイート|
|popular|人気のあるツイート（基準は不明）|
|mixed|recentとpopularの結果を合わせたもの|

<br/>

※ tweet_mode='extended'...140文字以上のツイートも取得可能になる

##### 使えそうなクエリ

- "島風 かわいい" - 島風 かわいいが含まれるツイートの検索
- 島風 OR 天津風 - 島風か天津風が含まれるツイートの検索
- 島風 -天津風 - 島風が含まれ天津風が含まれないツイートの検索
- #島風 - ハッシュタグ島風が含まれるツイートの検索
- from:user - ユーザーuserのツイートを検索
- to:user - 宛先がユーザーuserのツイートを検索
- @user - リプライ@userが含まれるツイートの検索
- 島風 since:2015-02-23 - 2015年2月23日以降の島風が含まれるツイートを最新から順に検索(search/tweetsでは1週間以上前のツイートは検索不可)
- 島風 until:2015-02-23 - 2015年2月23日以前の島風が含まれるツイートを最新から順に検索(search/tweetsでは1週間以上前のツイートは検索不可)
- 島風 :) - 島風が含まれ内容がポジティブなツイートの検索
- 島風 :( - 島風が含まれ内容がネガティブなツイートの検索
- 島風 ? - 島風が含まれ内容が疑問形なツイートの検索
- 島風 source:flantter - 島風が含まれFlantterからつぶやかれたツイートの検索
- 島風 lang:ja - 島風が含まれ日本語のツイートを検索(他en等)
- 島風 include:retweets - 島風が含まれるリツイートを含むツイートを検索
- 島風 exclude:retweets - 島風が含まれるリツイートを含まないツイートを検索
- 島風 exclude:nativeretweets - 島風が含まれるリツイートを含まないツイートを検索?
- 島風 list:user/listname - userのlistnameというリスト内で島風が含まれるツイートを検索
- 島風 geocode:37.78115,-122.39872,1mi - 島風を含み、緯度、経度、範囲を指定して検索(単位はkm,mi等)
- 島風 near:me - 島風を含み自分に近いところにいる人間のツイートを検索 (ドキュメントに書いてない)
- 島風 near:新潟 - 島風を含み新潟付近でつぶやかれたツイートの検索
- 島風 near:新潟 within:10km - 島風を含み新潟付近でつぶやかれたツイートの検索(範囲を10kmに指定、単位はkm,mi等)
- 島風 filter:links - 島風とリンクが含まれるツイートの検索
- 島風 filter:verified - 島風が含まれ認証されたアカウントからつぶやかれたツイートを検索
- 島風 filter:images - 島風と画像が含まれるツイートの検索
- 島風 filter:twimg - 島風とツイッターの画像が含まれるツイートの検索
- 島風 filter:videos - 島風と動画が含まれるツイートの検索
- 島風 filter:media - 島風とメディア(画像と動画)が含まれるツイートの検索
- 島風 filter:vine - 島風とVineが含まれるツイートの検索 (ドキュメントに書いてない)
- 島風 filter:news - 島風が含まれ、ニュースだと思われるツイートの検索 (ドキュメントに書いてない)
- 島風 filter:safe - 島風が含まれ、possibly_sensitiveフラグが0のツイートの検索？
- 島風 filter:periscope - 島風が含まれ、periscopeで配信しているツイートの検索
- 島風 filter:native_video - 島風が含まれ、periscope,vineまたはTwitterにアップロードされた動画の検索
- 島風 card_name:animated_gif - 島風が含まれ、GIFが含まれるツイートを検索 (ドキュメントに書いてない)
- 島風 min_retweets:100 - 島風が含まれリツイートが100以上のツイートを検索 (ドキュメントに書いてない)
- 島風 min_faves:100 - 島風が含まれお気に入りが100以上のツイートを検索 (ドキュメントに書いてない)
- 島風 min_replies:5 - 島風が含まれリプライが5以上のツイートを検索 (ドキュメントに書いてない)

In [28]:
#検索条件を元にツイートを抽出
tweets = tweepy.Cursor(api.search_tweets, q=search_word, tweet_mode='extended', result_type="mixed", lang='ja').items(req_tweet_cnt)

##### 取得できる時間データは世界時間なので日本時間に形式を変換

In [29]:
#世界時間を日本時間に変換する
def change_time_JST(time):
    utc = datetime(time.year, time.month,time.day, \
    time.hour,time.minute,time.second, tzinfo=timezone.utc)
    jst = utc.astimezone(pytz.timezone("Asia/Tokyo"))
    str_time = jst.strftime("%Y-%m-%d_%H:%M:%S")
    return str_time

##### Tweetデータを格納

In [30]:
# 取得したデータを格納する配列
tweet_data = []

# データを格納するイテレーション
for tweet in tweets:
    #ツイート時刻とユーザのアカウント作成時刻を日本時刻にする
    tweet_time = change_time_JST(tweet.created_at)
    create_account_time = change_time_JST(tweet.user.created_at)

    # リツイートとリプライを除外しながらデータを取得

    #tweet_dataの配列に取得したい情報を入れていく
    tweet_data.append([
        tweet.id,
        tweet_time,
        tweet.full_text,
        tweet.favorite_count,
        tweet.retweet_count,
        tweet.user.id,
        tweet.user.screen_name,
        tweet.user.name,
        # tweet.user.description,
        # tweet.user.friends_count,
        # tweet.user.followers_count,
        # create_account_time,
        # tweet.user.following,
        tweet.user.profile_image_url,
        tweet.user.profile_background_image_url,
        # tweet.user.url
    ])

##### 取り出したデータをpandasのDataFrameに変換

In [31]:
#CSVファイルに出力するときの列の名前を定義
columns=[
    'ツイートID',
    'ツイート時刻',
    'ツイート本文',
    'いいね数',
    'リツイート数',
    'ID',
    'ユーザー名',
    'アカウント名',
    # '自己紹介文',
    # 'フォロー数',
    # 'フォロワー数',
    # 'アカウント作成日時',
    # '自分のフォロー状況',
    'アイコン画像URL',
    'ヘッダー画像URL',
    # 'WEBサイト'
    ]

#tweet_dataのリストをpandasのDataFrameに変換
df = pd.DataFrame(tweet_data, columns=columns)

##### CSVファイルに出力する

In [32]:
#CSVファイルの名前を決める
file_name='searched_tweets.csv'

#CSVファイルを出力する
df.to_csv('./output/'+file_name,encoding='utf-8-sig',index=False)